In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/jangjungkeon/Study/main/Data/dataset/articlesapril.csv')
# print(df.head(3))
# print(df.count)
# print(df.columns)
# print(len(df.columns))

                  articleID  articleWordCount            byline documentType  \
0  5adf6684068401528a2aa69b               781    By JOHN BRANCH      article   
1  5adf653f068401528a2aa697               656  By LISA FRIEDMAN      article   
2  5adf4626068401528a2aa628              2427     By PETE WELLS      article   

                                            headline  \
0  Former N.F.L. Cheerleaders’ Settlement Offer: ...   
1  E.P.A. to Unveil a New Rule. Its Effect: Less ...   
2                            The New Noma, Explained   

                                            keywords  multimedia  newDesk  \
0  ['Workplace Hazards and Violations', 'Football...          68   Sports   
1  ['Environmental Protection Agency', 'Pruitt, S...          68  Climate   
2  ['Restaurants', 'Noma (Copenhagen, Restaurant)...          66   Dining   

   printPage              pubDate   sectionName  \
0          0  2018-04-24 17:16:49  Pro Football   
1          0  2018-04-24 17:11:21       Unk

In [2]:
# print(df['headline'].head())
# print(df.headline.values)
headline = []
headline.extend(list(df.headline.values))
# print(headline[:10])

# Unknown 값은 노이즈로 판단해 제거.
# print(len(headline))
headline = [n for n in headline if n != "Unknown"]
# print(len(headline))    # 1214
# print(headline[:10])

# 구둣점 제거, 소문자 처리 encode, decode, punctuation ---------
print("he하이llo 가a나b123다".encode("ascii", errors="ignore").decode())
from string import punctuation
print(", python.'".strip(punctuation))
print(", python.'".strip(punctuation + " "))

# -------------------------------------
def repre_func(s):
  s = s.encode("utf-8").decode("ascii", 'ignore')
  return "".join(c for c in s if c not in punctuation).lower()

text = [repre_func(s) for s in headline]
print(text[:10])

hello ab123
 python
python
['former nfl cheerleaders settlement offer 1 and a meeting with goodell', 'epa to unveil a new rule its effect less science in policymaking', 'the new noma explained', 'how a bag of texas dirt  became a times tradition', 'is school a place for selfexpression', 'commuter reprogramming', 'ford changed leaders looking for a lift its still looking', 'romney failed to win at utah convention but few believe hes doomed', 'chain reaction', 'he forced the vatican to investigate sex abuse now hes meeting with pope francis']


In [3]:
# 단어 집합 생성
from keras_preprocessing.text import Tokenizer
tok = Tokenizer()
tok.fit_on_texts(text)
print(tok.word_index)
vocab_size = len(tok.word_index) + 1
print('vocab_size : ', vocab_size)

sequences = list()
for line in text:
  enc = tok.texts_to_sequences([line])[0]
  for i in range(1, len(enc)):
    se = enc[: i+1]
    sequences.append(se)

print(sequences[:11])

index_to_word = {}
for key, value in tok.word_index.items():
  index_to_word[value] = key
print(index_to_word[1])
print(index_to_word[150])

max_len = max(len(i) for i in sequences)
print('max_len : ', max_len)

# padding 처리하기 : 문자길이를 동일하게 맞추기위해.
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

import numpy as np
sequences = np.array(sequences)
x = sequences[:, :-1]   # feature
y = sequences[:, -1]   # label
print(x[:3])
print(y[:3])

# label one-hot encoding
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)
print(y[0])


{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'in': 5, 'for': 6, 'and': 7, 'is': 8, 'on': 9, 'with': 10, 'trump': 11, 'as': 12, 'at': 13, 'new': 14, 'how': 15, 'from': 16, 'it': 17, 'an': 18, 'that': 19, 'be': 20, 'season': 21, 'us': 22, 'you': 23, 'its': 24, 'what': 25, 'episode': 26, 'can': 27, 'your': 28, 'not': 29, 'he': 30, 'now': 31, 'his': 32, 'are': 33, 'teaching': 34, 'war': 35, 'out': 36, 'no': 37, 'was': 38, 'by': 39, 'trumps': 40, 'has': 41, 'over': 42, 'may': 43, 'into': 44, 'why': 45, 'more': 46, 'we': 47, 'who': 48, 'about': 49, 'recap': 50, 'activities': 51, '1': 52, 'just': 53, 'do': 54, 'women': 55, 'when': 56, 'syria': 57, 'trade': 58, 'i': 59, '2': 60, 'or': 61, 'will': 62, 'this': 63, 'have': 64, 'president': 65, 'but': 66, 'home': 67, 'up': 68, 'long': 69, 'one': 70, 'off': 71, 'facebook': 72, 'house': 73, 'gop': 74, 'our': 75, 'case': 76, 'they': 77, 'life': 78, 'end': 79, 'right': 80, 'some': 81, 'big': 82, 'dead': 83, 'power': 84, 'say': 85, 'white': 86, 'after': 87, 's

In [4]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_len -1))
model.add(LSTM(128, activation='tanh'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=50, verbose=2, batch_size=32)
print(model.evaluate(x, y))


Epoch 1/50
244/244 - 5s - loss: 7.6374 - accuracy: 0.0273
Epoch 2/50
244/244 - 5s - loss: 7.0820 - accuracy: 0.0323
Epoch 3/50
244/244 - 4s - loss: 6.9173 - accuracy: 0.0370
Epoch 4/50
244/244 - 4s - loss: 6.7634 - accuracy: 0.0431
Epoch 5/50
244/244 - 4s - loss: 6.6021 - accuracy: 0.0468
Epoch 6/50
244/244 - 4s - loss: 6.4289 - accuracy: 0.0516
Epoch 7/50
244/244 - 3s - loss: 6.2470 - accuracy: 0.0575
Epoch 8/50
244/244 - 5s - loss: 6.0615 - accuracy: 0.0613
Epoch 9/50
244/244 - 4s - loss: 5.8739 - accuracy: 0.0678
Epoch 10/50
244/244 - 4s - loss: 5.6941 - accuracy: 0.0730
Epoch 11/50
244/244 - 4s - loss: 5.5186 - accuracy: 0.0809
Epoch 12/50
244/244 - 7s - loss: 5.3548 - accuracy: 0.0841
Epoch 13/50
244/244 - 10s - loss: 5.1945 - accuracy: 0.0927
Epoch 14/50
244/244 - 11s - loss: 5.0431 - accuracy: 0.0980
Epoch 15/50
244/244 - 6s - loss: 4.8983 - accuracy: 0.1077
Epoch 16/50


KeyboardInterrupt: 

In [ ]:
# 텍스트 생성하기.
def sentence_gen(model, t, current_word, n):
  init_word = current_word
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
    result = np.argmax(model.predict(encoded), axis=-1)
    # print(result)
    for word, index in t.word_index.items():
      # print('word:', word, ', index: ',  index)
      if index == result:
        break
    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word    # 예측한 단어를 문장에 저장.

  sentence = init_word + sentence
  return sentence

print(sentence_gen(model, tok, 'i', 10))
print(sentence_gen(model, tok, 'how', 10))
print(sentence_gen(model, tok, 'how', 100))
print(sentence_gen(model, tok, 'good', 200))